<a href="https://colab.research.google.com/github/slogen/snippets/blob/main/SimpleConfigTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
### Sample JSON decoded

config = {
    "w": "$abspath/bar",
    "x": 1,
    "y": ["a","b","c"],
    "z": {
        "z1": 1,
        "z2": ["z2.0", "z2.1"],
        "z3": {},
        "q": "some $abspath stuff"
    }
}

In [78]:
### Simple replacement

from string import Template
def g(ctx):
  def f(c):
    if    isinstance(c, dict): return dict((k,f(v)) for (k,v) in c.items())
    elif  isinstance(c, list): return [f(v) for v in c]
    elif  isinstance(c, str): return Template(c).substitute(**ctx)
    else: return c
  return f

ctx = {"abspath": "XXX"}
x = g(ctx)(config)

# Does translation
assert x['w'] == Template(config['w']).substitute(**ctx)
assert x['z']['q'] == Template(config['z']['q']).substitute(**ctx)
# Lookup through chain works
assert x['z']['z2'][0] == config['z']['z2'][0]


In [79]:
### "Inheriting" configuration

from typing_extensions import ChainMap
def g(ctx):
  def recurse(c, a):
    if isinstance(c, dict): 
      a2 = a.new_child()
      for (k,v) in c.items():
        a2[k] = recurse(v, a2)
      return a2
    elif  isinstance(c, list): return [recurse(v, a) for v in c]
    elif  isinstance(c, str): return Template(c).substitute(**ctx)
    else: return c
  def r(c): return recurse(c,ChainMap(ctx))
  return r

ctx = {"abspath": "XXX"}
x = g(ctx)(config)

# Does translation
assert x['w'] == Template(config['w']).substitute(**ctx)
assert x['z']['q'] == Template(config['z']['q']).substitute(**ctx)
# Lookup through chain works
assert x['z']['z2'][0] == config['z']['z2'][0]
# Falls back to lookup further up chain
assert x['z']['x'] == config['x']